# 2. Entendimiento de los Datos

In [1]:
## Importamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from fuzzywuzzy import fuzz
#from fuzzywuzzy import process
import re

In [2]:
# Load Datlas Data
data_datlas = pd.read_excel('../data/data_datlas.xlsx')

In [3]:
# Load Weather Data
# Source: https://www.ncei.noaa.gov/access/past-weather/Monterrey%2C%20Nuevo%20Le%C3%B3n, 
# tomemos la precipitación de Monterrey como si fuera del estado.
data_weather = pd.read_csv('../data/data_weather.csv')
data_weather.head(3)

,Date,TAVG (Degrees Fahrenheit),TMAX (Degrees Fahrenheit),TMIN (Degrees Fahrenheit),PRCP (Inches),SNOW (Inches),SNWD (Inches)
0,15/03/1977,76.0,91.0,57.0,0.0,NaN,NaN
1,18/03/1977,81.0,NaN,NaN,0.0,NaN,NaN
2,21/03/1977,71.0,NaN,NaN,0.0,NaN,NaN


## 2.1 Contexto de los Datos

In [4]:
data_datlas.head()

,FOLIO_ID,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,ESTADO,CIUDAD_APROXIMADA,DATE
0,D00001,22.768507,-102.567896,98000,*EP 2 Boulevard Adolfo López Mateos,CERRADA DE CASA BLANCA,COLISION Y/O VUELCO,Camión,BLANCO,2016,Bajo,NaN,2018,1,11,JUEVES,15,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,2018-01-11
1,D00002,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018,1,25,JUEVES,10,NUEVO LEÓN,JUAREZ,2018-01-25
2,D00003,20.679917,-103.368657,44600,AV MEXICO,2 RIOS,COLISION Y/O VUELCO,Auto,BLANCO,2005,Sin daño,Trasero,2018,1,15,LUNES,9,NUEVO LEÓN,GUADALUPE,2018-01-15
3,D00004,25.718027,-100.249990,66480,Avenida Conductores,Conductores Infonavit,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018,1,19,VIERNES,9,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,2018-01-19
4,D00005,19.266303,-99.331791,\N,UBERP3,Cuajimalpa,HUELGAS Y ALBOROTOS,Auto,NEGRO,2016,Medio,NaN,2018,1,3,MIERCOLES,16,NUEVO LEÓN,MONTERREY,2018-01-03


In [5]:
# Información general del dataframe
data_datlas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116011 entries, 0 to 116010
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   FOLIO_ID             116011 non-null  object        
 1   LAT                  113854 non-null  float64       
 2   LONG                 113854 non-null  float64       
 3   CODIGO POSTAL        115926 non-null  object        
 4   CALLE                116011 non-null  object        
 5   COLONIA              115921 non-null  object        
 6   CAUSA SINIESTRO      116011 non-null  object        
 7   TIPO VEHICULO        115917 non-null  object        
 8   COLOR                115852 non-null  object        
 9   MODELO_VEHICULO      115925 non-null  object        
 10  NIVEL DAÑO VEHICULO  115088 non-null  object        
 11  PUNTO DE IMPACTO     104932 non-null  object        
 12  AÑO                  116011 non-null  int64         
 13  MES           

Tenemos un dataframe con 116,011 registros y 20 columnas, toda la data es referente a registros historicos sobre choques y siniestros de una aseguradora, las variables que tenemos son:

**TIPOS DE DATO CATEGORICOS**

- **FOLIO_ID (ordinal):** Es el id del registro.

- **CODIGO POSTAL (nominal):** Designa un región geográfica.

- **CALLE (nominal):** Representa la calle del siniestro.

- **COLONIA (nominal):** Representa la colonia del incidente.

- **CAUSA SINIESTRO (Nominal):** Causa principal del siniestro.

- **TIPO VEHICULO (nominal):** Auto, Camión, Motocicleta, etc.

- **COLOR (nominal):** Color del vehiculo.

- **MODELO_VEHICULO (ordinal):** - Año del modelo de vehiculo. 

- **NIVEL DAÑO DEL VEHICULO (nominal):** Nivel de daño del vehiculo.

- **PUNTO DE IMPACTO (nominal):** Punto de impacto en el siniestro.

- **DIA (nominal):** Dia de la semana en que se dio el impacto.

- **ESTADO (nominal):** Estado dentro de México donde se dio el impacto.

- **CIUDAD_APROXIMADA (nominal):** Ciudad del Estado donde ocurrio el siniestro.

**TIPOS DE DATO NÚMERICO**

- **LAT (continuo):** Coordenada geográfica de Latitud.

- **LONG (continuo):** Coordenada geográfica de Longitud.

- **AÑO (discreto):** Año en que se dio el impacto.

- **MES (discreto):** Mes en que se dio el impacto.

- **DIA NÚMERO (discreto):** Dia de la semana en que se dio el impacto.

- **HORA (discreto):** Hora en que se dio el impacto.

**TIPO DE DATO DATETIME**

- **DATE (datetime - categorico ordinal):** Representa la fecha del incidente.

## 2.2 Revisar el Contexto **y** Calidad de los Datos

In [6]:
# Estadistica descriptiva
data_datlas.describe()

,LAT,LONG,AÑO,MES,DÍA NUMERO,HORA,DATE
count,113854.000000,113854.000000,116011.000000,116011.000000,116011.000000,116011.000000,116011
mean,23.526022,-91.821885,2016.678479,6.005560,15.595797,13.824836,2017-02-18 14:23:26.783839488
min,-13.520364,-117.019350,2016.000000,1.000000,1.000000,0.000000,2016-01-01 00:00:00
25%,25.653531,-100.346878,2016.000000,3.000000,8.000000,10.000000,2016-07-10 00:00:00
50%,25.686510,-100.289794,2017.000000,6.000000,15.000000,14.000000,2017-01-26 00:00:00
75%,25.734853,-100.214012,2017.000000,9.000000,23.000000,18.000000,2017-09-23 00:00:00
max,47.827586,35.167756,2018.000000,12.000000,31.000000,23.000000,2018-06-30 00:00:00
std,7.145716,27.876454,0.720202,3.406765,8.745570,4.741398,NaN


In [7]:
# Buscar valores faltaltes en las filas y columnas
def buscar_valores_faltantes(data, eje):
  valores_faltantes = data.isnull().any(axis=eje).sum()
  if valores_faltantes > 0:
    print(f"Hay {valores_faltantes} features con valores faltantes en el axis {eje}.")
  else:
    print(f"No hay valores faltantes en el axis {eje}")

buscar_valores_faltantes(data_datlas, 1)

Hay 13034 features con valores faltantes en el axis 1.


In [8]:
## ¿Qué columnas tienen valores faltantes?
valores_faltantes_columnas=data_datlas.isnull().any(axis=0)
print(valores_faltantes_columnas)
print("-------------------------------")
buscar_valores_faltantes(data_datlas, 0)

FOLIO_ID               False
LAT                     True
LONG                    True
CODIGO POSTAL           True
CALLE                  False
COLONIA                 True
CAUSA SINIESTRO        False
TIPO VEHICULO           True
COLOR                   True
MODELO_VEHICULO         True
NIVEL DAÑO VEHICULO     True
PUNTO DE IMPACTO        True
AÑO                    False
MES                    False
DÍA NUMERO             False
DIA                    False
HORA                   False
ESTADO                 False
CIUDAD_APROXIMADA      False
DATE                   False
dtype: bool
-------------------------------
Hay 9 features con valores faltantes en el axis 0.


Dado que tenemos 13,034 filas con valores faltantes en las columnas **LAT, LONG, CODIGO POSTAL, COLONIA, TIPO VEHICULO, COLOR, MODELO, NIVEL DE DAÑO y PUNTO DE IMPACTO**, a partir de aquí tomare la decisión de eliminar todas estás filas del dataframe porque son valores que generan ruido al análisis.

Cabe mencionar que en otras circunstancias este tipo de valores faltantes se podría conseguir con ayuda de otra base de datos o con una API de geolocalización en el caso de los valores geográficos.

In [9]:
## Eliminamos las filas con valores faltantes.
data_datlas=data_datlas.dropna()
## Buscamos filas con registros faltanes
buscar_valores_faltantes(data_datlas, 1)
## Buscamos columnas con registros faltantes
buscar_valores_faltantes(data_datlas, 0)

No hay valores faltantes en el axis 1
No hay valores faltantes en el axis 0


In [10]:
# Buscando filas con valores duplicadas
duplicados = data_datlas.duplicated()
print(len(data_datlas[duplicados]), "filas duplicadas.")

0 filas duplicadas.


In [11]:
## Veamos nuevamente la información del dataframe
print(data_datlas.info())
data_datlas.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 102977 entries, 1 to 116010
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   FOLIO_ID             102977 non-null  object        
 1   LAT                  102977 non-null  float64       
 2   LONG                 102977 non-null  float64       
 3   CODIGO POSTAL        102977 non-null  object        
 4   CALLE                102977 non-null  object        
 5   COLONIA              102977 non-null  object        
 6   CAUSA SINIESTRO      102977 non-null  object        
 7   TIPO VEHICULO        102977 non-null  object        
 8   COLOR                102977 non-null  object        
 9   MODELO_VEHICULO      102977 non-null  object        
 10  NIVEL DAÑO VEHICULO  102977 non-null  object        
 11  PUNTO DE IMPACTO     102977 non-null  object        
 12  AÑO                  102977 non-null  int64         
 13  MES                

,FOLIO_ID,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,ESTADO,CIUDAD_APROXIMADA,DATE
1,D00002,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018,1,25,JUEVES,10,NUEVO LEÓN,JUAREZ,2018-01-25
2,D00003,20.679917,-103.368657,44600,AV MEXICO,2 RIOS,COLISION Y/O VUELCO,Auto,BLANCO,2005,Sin daño,Trasero,2018,1,15,LUNES,9,NUEVO LEÓN,GUADALUPE,2018-01-15
3,D00004,25.718027,-100.249990,66480,Avenida Conductores,Conductores Infonavit,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018,1,19,VIERNES,9,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,2018-01-19


In [12]:
## Reseteamos el indice
data_datlas.reset_index()
data_datlas.head()

,FOLIO_ID,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,ESTADO,CIUDAD_APROXIMADA,DATE
1,D00002,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018,1,25,JUEVES,10,NUEVO LEÓN,JUAREZ,2018-01-25
2,D00003,20.679917,-103.368657,44600,AV MEXICO,2 RIOS,COLISION Y/O VUELCO,Auto,BLANCO,2005,Sin daño,Trasero,2018,1,15,LUNES,9,NUEVO LEÓN,GUADALUPE,2018-01-15
3,D00004,25.718027,-100.249990,66480,Avenida Conductores,Conductores Infonavit,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018,1,19,VIERNES,9,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,2018-01-19
5,D00006,19.706476,-99.336598,54650,DUQUE DE BRAGANZA No.331,Santiago el Bajo** REL DE PALMAS,COLISION Y/O VUELCO,Auto,BLANCO,2007,Sin daño,Trasero,2018,1,11,JUEVES,21,NUEVO LEÓN,GENERAL ZUAZUA,2018-01-11
7,D00008,25.723590,-100.198780,67117,Boulevard Acapulco,REAL DE SAN MIGUEL,COLISION Y/O VUELCO,Auto,BLANCO,2000,Sin daño,Frontal,2018,1,29,LUNES,17,NUEVO LEÓN,GUADALUPE,2018-01-29


## 2.3 Limpieza

In [13]:
# Obtenemos la cantidad de valores únicos por columna
valores_unicos_por_columna = data_datlas.nunique()

# Imprimir la cantidad de valores únicos por columna
print(valores_unicos_por_columna)

FOLIO_ID               102977
LAT                     77745
LONG                    78326
CODIGO POSTAL             810
CALLE                   43030
COLONIA                 18837
CAUSA SINIESTRO             7
TIPO VEHICULO               5
COLOR                    2341
MODELO_VEHICULO           102
NIVEL DAÑO VEHICULO         5
PUNTO DE IMPACTO           13
AÑO                         3
MES                        12
DÍA NUMERO                 31
DIA                         7
HORA                       24
ESTADO                      1
CIUDAD_APROXIMADA          73
DATE                      911
dtype: int64


Los valores unicos de cada columna nos dan información importante, de aquí sabemos que debemos corregir al menos el COLOR, ya que es muy poco probable que sean 2341 colores diferentes, así que vamos a comenzar a realizar limpieza columna por columna.

In [14]:
# Eliminamos la columna FOLIO_ID ya que no nos aporta información relevante en este punto.
data_datlas=data_datlas.drop('FOLIO_ID', axis=1)
data_datlas.columns

Index(['LAT', 'LONG', 'CODIGO POSTAL', 'CALLE', 'COLONIA', 'CAUSA SINIESTRO',
       'TIPO VEHICULO', 'COLOR', 'MODELO_VEHICULO', 'NIVEL DAÑO VEHICULO',
       'PUNTO DE IMPACTO', 'AÑO', 'MES', 'DÍA NUMERO', 'DIA', 'HORA', 'ESTADO',
       'CIUDAD_APROXIMADA', 'DATE'],
      dtype='object')

In [15]:
# En cuanto a latitud y longitud, podemos filtrar en un cuadro de coordenadas
# Esto mediante nos asegura que los datos son de NUEVO LEON
# Para ello, usare las coordenadas de la pagina del INEGI: https://www.inegi.org.mx/app/mapa/espacioydatos/default.aspx
lat_min = 23.1535
lat_max = 27.8410
long_min = -101.2830
long_max = -98.4045

data_datlas = data_datlas[data_datlas['LAT'] < lat_max]
data_datlas = data_datlas[data_datlas['LAT'] > lat_min]
data_datlas = data_datlas[data_datlas['LONG'] < long_max]
data_datlas = data_datlas[data_datlas['LONG'] > long_min]
print("-------------------------------------------------------------")
print(data_datlas.columns)

-------------------------------------------------------------
Index(['LAT', 'LONG', 'CODIGO POSTAL', 'CALLE', 'COLONIA', 'CAUSA SINIESTRO',
       'TIPO VEHICULO', 'COLOR', 'MODELO_VEHICULO', 'NIVEL DAÑO VEHICULO',
       'PUNTO DE IMPACTO', 'AÑO', 'MES', 'DÍA NUMERO', 'DIA', 'HORA', 'ESTADO',
       'CIUDAD_APROXIMADA', 'DATE'],
      dtype='object')


In [16]:
## Para el codigo postal, deben ser codigos de 5 digitos, así que vamos a extraer solo aquellos que cumplan con la condición
data_datlas['CODIGO POSTAL'] = data_datlas['CODIGO POSTAL'].str.extract(r'(\d{5})')
# Eliminamos los NAs
data_datlas = data_datlas.dropna()

In [17]:
## Veamos como estamos en calles, colonias y los colores
print('Cantidad de calles', len(data_datlas['CALLE'].unique()))
print('Cantidad de colonias', len(data_datlas['COLONIA'].unique()))
print('Cantidad de calles', len(data_datlas['COLOR'].unique()))

Cantidad de calles 38228
Cantidad de colonias 16499
Cantidad de calles 2145


In [18]:
## Hacemos la columna en palabras mayusculas
data_datlas['CALLE'] = data_datlas['CALLE'].str.upper()
data_datlas['COLONIA'] = data_datlas['COLONIA'].str.upper()
data_datlas['COLOR'] = data_datlas['COLOR'].str.upper()

In [19]:
# Ver cuales calles son mas populares
data_datlas['CALLE'].value_counts().head(25)

CALLE
AVENIDA EUGENIO GARZA SADA            885
AVENIDA MIGUEL ALEMÁN                 802
CARRETERA NACIONAL                    590
AVENIDA LÁZARO CÁRDENAS               584
AVENIDA ALFONSO REYES                 565
AVENIDA REVOLUCIÓN                    501
ABRAHAM LINCOLN                       474
AV. MIGUEL ALEMÁN                     464
AV. PASEO DE LOS LEONES               453
AVENIDA CONSTITUCIÓN                  441
AVENIDA PASEO DE LOS LEONES           435
AVENIDA ABRAHAM LINCOLN               388
AV. ADOLFO RUIZ CORTÍNES              361
AVENIDA UNIVERSIDAD                   304
ALFONSO REYES                         303
LIBRAMIENTO NORESTE                   275
AVENIDA DR JOSÉ ELEUTERIO GONZÁLEZ    274
AVENIDA ELOY CAVAZOS                  261
MANUEL BARRAGÁN                       253
GONZALITOS                            251
BENITO JUÁREZ                         245
UNIVERSIDAD                           243
MIGUEL ALEMAN                         243
ELOY CAVAZOS                

In [20]:
# Ver cuales calles son mas populares
data_datlas['COLONIA'].value_counts().head(25)

COLONIA
MONTERREY CENTRO                     3882
DEL VALLE                            2234
VISTA HERMOSA                        1684
CENTRO                               1586
FIERRO                               1580
\N                                   1449
HOMERO SEPÚLVEDA                     1066
BOSQUES DEL NOGALAR                  1032
PRIVADAS DE ANÁHUAC SECTOR INGLÉS     945
FRANCISCO GARCÍA NARANJO (INDECO)     924
PLAZA SAN ANTONIO                     900
MITRAS CENTRO                         823
LOS FAISANES SECTOR EL DORADO         750
RESIDENCIAL CUMBRES                   691
JARDÍN DE LAS PUENTES                 622
VILLA ALTA                            603
BOSQUES DE SANTA ROSA                 594
CROC INFONAVIT                        592
CONTRY                                584
CADEREYTA JIMÉNEZ CENTRO              560
OBISPADO                              499
SIN DATO                              468
VALLE DE LAS BRISAS                   459
BOSQUES DE LA SILLA       

In [21]:
# Ver cuales calles son mas populares
data_datlas['COLOR'].value_counts().head(25)

COLOR
BLANCO        27951
GRIS          15931
ROJO           8609
PLATA          6476
NEGRO          5881
AZUL           5638
ARENA          3049
VERDE          2369
BLANCA         1966
AMARILLO       1622
GUINDA         1119
VINO            973
\N              845
DORADO          705
CAFE            676
GRIS PLATA      640
GUINDO          446
NEGRA           431
ROJA            385
NARANJA         301
BCO             259
ANARANJADO      201
BEIGE           197
CELESTE         145
BLANCO CON      138
Name: count, dtype: int64

In [22]:
##  Reemplazamos valores de filas con valores similares de los más comunes

def remplazar_valores(df, columna, valor_a_remplazar, valor_remplazo):
    df[columna] = df[columna].replace(valor_a_remplazar, valor_remplazo)

remplazar_valores(data_datlas, 'COLOR', r'BLANCA', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCODE', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO14', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO 16', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO16', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'2014 BCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'2015 BCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCODE ', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/VER ', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/NEG ', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/13', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/14', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/15', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/16', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'2014 BLANCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'2014BLANCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'2015 BLANCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'2016 BLANCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO2013', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO2014', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO2015', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO2016', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO2017', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/2015', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/ROJO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO CON', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/VERDE', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/AZUL', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/AZU', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BALNCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'2016BLANCO', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO/ROJ', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO 15', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'BLANCO2012', 'BLANCO')
remplazar_valores(data_datlas, 'COLOR', r'GRIS PLATA', 'GRIS')
remplazar_valores(data_datlas, 'COLOR', r'GRIS OBSCU', 'GRIS')
remplazar_valores(data_datlas, 'COLOR', r'GRIS OSCUR', 'GRIS')
remplazar_valores(data_datlas, 'COLOR', r'GRIS METALICO', 'GRIS')
remplazar_valores(data_datlas, 'COLOR', r'GRIS OXFOR', 'GRIS')
remplazar_valores(data_datlas, 'COLOR', r'GRIS2016', 'GRIS')
remplazar_valores(data_datlas, 'COLOR', r'GRIS2017', 'GRIS')
remplazar_valores(data_datlas, 'COLOR', r'GRIS/NEGRO', 'GRIS')    
remplazar_valores(data_datlas, 'COLOR', r'GRIS ACERO', 'GRIS')
remplazar_valores(data_datlas, 'COLOR', r'ROJA', 'ROJO')
remplazar_valores(data_datlas, 'COLOR', r'ROJO2016', 'ROJO')
remplazar_valores(data_datlas, 'COLOR', r'ROJO2017', 'ROJO')
remplazar_valores(data_datlas, 'COLOR', r'ANARANJADO', 'NARANJA')
remplazar_valores(data_datlas, 'COLOR', r'NEGRA', 'NEGRO')
remplazar_valores(data_datlas, 'COLOR', r'AZUL CIELO', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'AZUL REY', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'AZUL MARIN', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'AZUL/BLANCO', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'AZUL/BLANC', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'AZUL GRIS', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'AZUL CLARO', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'AZULO', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'AZUL METALICO', 'AZUL')
remplazar_valores(data_datlas, 'COLOR', r'GUINDA', 'GUINDO')
remplazar_valores(data_datlas, 'COLOR', r'GINDA', 'GUINDO')
remplazar_valores(data_datlas, 'COLOR', r'VINO', 'GUINDO')
remplazar_valores(data_datlas, 'COLOR', r'GINDO', 'GUINDO')
remplazar_valores(data_datlas, 'COLOR', r'PLATEADO', 'PLATA')
remplazar_valores(data_datlas, 'COLOR', r'PLATEADA', 'PLATA')
remplazar_valores(data_datlas, 'COLOR', r'DORADA', 'DORADO')
remplazar_valores(data_datlas, 'COLOR', r'AMARILLA', 'AMARILLO')

In [23]:
# Vamos con las calles
data_datlas['CALLE'] = data_datlas['CALLE'].apply(lambda x: re.sub('AVENIDA', 'AV.', str(x)))
remplazar_valores(data_datlas, 'CALLE', r'UNIVERSIDAD', 'AV. UNIVERSIDAD')
remplazar_valores(data_datlas, 'CALLE', r'ABRAHAM LINCOLN', 'AV. ABRAHAM LINCOLN')
remplazar_valores(data_datlas, 'CALLE', r'ALFONSO REYES', 'AV. ALFONSO REYES')
remplazar_valores(data_datlas, 'CALLE', r'MIGUEL ALEMAN', 'AV. MIGUEL ALEMÁN')
remplazar_valores(data_datlas, 'CALLE', r'LÁZARO CÁRDENAS', 'AV. LÁZARO CÁRDENAS')
remplazar_valores(data_datlas, 'CALLE', r'MANUEL BARRAGÁN', 'AV. MANUEL LUIS BARRAGÁN')
remplazar_valores(data_datlas, 'CALLE', r'AV. MANUEL L. BARRAGÁN', 'AV. MANUEL LUIS BARRAGÁN')
remplazar_valores(data_datlas, 'CALLE', r'AV UNIVERSIDAD', 'AV. UNIVERSIDAD')
remplazar_valores(data_datlas, 'CALLE', r'AV. ADOLFO RUÍZ CORTINES', 'AV. ADOLFO RUIZ CORTÍNES')
remplazar_valores(data_datlas, 'CALLE', r'AV RUIZ CORTINEZ', 'AV. ADOLFO RUIZ CORTÍNES')
remplazar_valores(data_datlas, 'CALLE', r'ELOY CAVAZOS', 'AV. ELOY CAVAZOS')
remplazar_valores(data_datlas, 'CALLE', r'AV. ADOLFO RUIZ CORTINES', 'AV. ADOLFO RUIZ CORTÍNES')
remplazar_valores(data_datlas, 'CALLE', r'AV. ADOLFO RUIZ CORTINEZ', 'AV. ADOLFO RUIZ CORTÍNES')
remplazar_valores(data_datlas, 'CALLE', r'PABLO LIVAS', 'AV. PABLO LIVAS')
remplazar_valores(data_datlas, 'CALLE', r'LAZARO CARDENAS', 'AV. LÁZARO CÁRDENAS')
remplazar_valores(data_datlas, 'CALLE', r'AV LAZARO CARDENAS', 'AV. LÁZARO CÁRDENAS')
remplazar_valores(data_datlas, 'CALLE', r'ADOLFO LÓPEZ MATEOS', 'AV. ADOLFO LÓPEZ MATEOS')
remplazar_valores(data_datlas, 'CALLE', r'LOPEZ MATEOS', 'AV. ADOLFO LÓPEZ MATEOS')
remplazar_valores(data_datlas, 'CALLE', r'CARRETERA MIGUEL ALEMÁN', 'AV. MIGUEL ALEMÁN')
remplazar_valores(data_datlas, 'CALLE', r'MIGUEL ALEMÁN', 'AV. MIGUEL ALEMÁN')
remplazar_valores(data_datlas, 'CALLE', r'I. MORONES PRIETO', 'AV. IGNACIO MORONES PRIETO')
remplazar_valores(data_datlas, 'CALLE', r'MORONES PRIETO', 'AV. IGNACIO MORONES PRIETO')
remplazar_valores(data_datlas, 'CALLE', r'AV MORONES PRIETO', 'AV. IGNACIO MORONES PRIETO')
remplazar_valores(data_datlas, 'CALLE', r'AV. I. MORONES PRIETO', 'AV. IGNACIO MORONES PRIETO')
remplazar_valores(data_datlas, 'CALLE', r'AV. DOCTOR IGNACIO MORONES PRIETO', 'AV. IGNACIO MORONES PRIETO')
remplazar_valores(data_datlas, 'CALLE', r'AV. IGNACION MORONES PRIETO', 'AV. IGNACIO MORONES PRIETO')
remplazar_valores(data_datlas, 'CALLE', r'EUGENIO GARZA SADA', 'AV. EUGENIO GARZA SADA')
remplazar_valores(data_datlas, 'CALLE', r'AV. EUGENIO GARZA SADA SUR', 'AV. EUGENIO GARZA SADA')
remplazar_valores(data_datlas, 'CALLE', r'AV EUGENIO GARZA SADA', 'AV. EUGENIO GARZA SADA')
remplazar_valores(data_datlas, 'CALLE', r'BENITO JUÁREZ', 'AV. BENITO JUÁREZ')
remplazar_valores(data_datlas, 'CALLE', r'BENITO JUAREZ', 'AV. BENITO JUÁREZ')
remplazar_valores(data_datlas, 'CALLE', r'MAESTRO ISRAEL CAVAZOS GARZA', 'AV. MAESTRO ISRAEL CAVAZOS GARZA')
remplazar_valores(data_datlas, 'CALLE', r'AV. ADOLFO RUIZ CORTÍNEZ', 'AV. ADOLFO RUIZ CORTÍNES')
remplazar_valores(data_datlas, 'CALLE', r'RÓMULO GARZA', 'AV. RÓMULO GARZA')
remplazar_valores(data_datlas, 'CALLE', r'AV ROMULO GARZA', 'AV. RÓMULO GARZA')
remplazar_valores(data_datlas, 'CALLE', r'ROMULO DÍAZ DE LA VEGA 1870', 'AV. RÓMULO GARZA')
remplazar_valores(data_datlas, 'CALLE', r'CONSTITUCION', 'AV. CONSTITUCIÓN')
remplazar_valores(data_datlas, 'CALLE', r'CONSTITUCIÓN', 'AV. CONSTITUCIÓN')
remplazar_valores(data_datlas, 'CALLE', r'AV CONSTITUCION', 'AV. CONSTITUCIÓN')
remplazar_valores(data_datlas, 'CALLE', r'AV. GÓMEZ MORÍN', 'AV. MANUEL GÓMEZ MORÍN')
remplazar_valores(data_datlas, 'CALLE', r'AV MIGUEL ALEMAN', 'AV. MIGUEL ALEMÁN')
remplazar_valores(data_datlas, 'CALLE', r'AV. MIGUEL ALEMÃ¡N', 'AV. MIGUEL ALEMÁN')
remplazar_valores(data_datlas, 'CALLE', r'AV. LEONES', 'AV. PASEO DE LOS LEONES')
remplazar_valores(data_datlas, 'CALLE', r'AV PASEO DE LOS LEONES', 'AV. PASEO DE LOS LEONES')
remplazar_valores(data_datlas, 'CALLE', r'PASEO DE LOS LEONES', 'AV. PASEO DE LOS LEONES')
remplazar_valores(data_datlas, 'CALLE', r'FIDEL VELAZQUEZ', 'AV. FIDEL VELÁZQUEZ')
remplazar_valores(data_datlas, 'CALLE', r'GONZALITOS', 'AV. GONZALITOS')
remplazar_valores(data_datlas, 'CALLE', r'AV GONZALITOS', 'AV. GONZALITOS')
remplazar_valores(data_datlas, 'CALLE', r'AV REVOLUCION', 'AV. REVOLUCIÓN')
remplazar_valores(data_datlas, 'CALLE', r'REVOLUCION', 'AV. REVOLUCIÓN')
remplazar_valores(data_datlas, 'CALLE', r'PLUTARCO ELIAS CALLES', 'AV. PLUTARCO ELIAS CALLES')
remplazar_valores(data_datlas, 'CALLE', r'AV. LICENCIADO RAÚL SALINAS LOZANO', 'AV. RAÚL SALINAS LOZANO')
remplazar_valores(data_datlas, 'CALLE', r'RAUL SALINAS', 'AV. RAÚL SALINAS LOZANO')
remplazar_valores(data_datlas, 'CALLE', r'RAUL SALINAS LOZANO', 'AV. RAÚL SALINAS LOZANO')
remplazar_valores(data_datlas, 'CALLE', r'BOULEVARD LICENCIADO GUSTAVO DÍAZ ORDAZ', 'BLV. GUSTAVO DÍAZ ORDAZ')
remplazar_valores(data_datlas, 'CALLE', r'CHAPULTEPEC', 'AV. CHAPULTEPEC')
remplazar_valores(data_datlas, 'CALLE', r'AV CHAPULTEPEC', 'AV. CHAPULTEPEC')
remplazar_valores(data_datlas, 'CALLE', r'CUAUHTÉMOC', 'AV. CUAUHTÉMOC')
remplazar_valores(data_datlas, 'CALLE', r'CUAUHTEMOC', 'AV. CUAUHTÉMOC')
remplazar_valores(data_datlas, 'CALLE', r'CONCORDIA', 'AV. CONCORDIA')
remplazar_valores(data_datlas, 'CALLE', r'DE LA CONCORDIA', 'AV. CONCORDIA')
remplazar_valores(data_datlas, 'CALLE', r'VENUSTIANO CARRANZA', 'AV. VENUSTIANO CARRANZA')
remplazar_valores(data_datlas, 'CALLE', r'ROMULO GARZA', 'AV. RÓMULO GARZA')
remplazar_valores(data_datlas, 'CALLE', r'AV LAS TORRES', 'AV. LAS TORRES')

In [24]:
## Veamos como estamos despues de la limpieza
print('Cantidad de calles', len(data_datlas['CALLE'].unique()))
print('Cantidad de colonias', len(data_datlas['COLONIA'].unique()))
print('Cantidad de colores', len(data_datlas['COLOR'].unique()))

Cantidad de calles 34817
Cantidad de colonias 13773
Cantidad de colores 1773


In [25]:
## Eliminamos las filas que contenga el simbolo \N
data_datlas = data_datlas[~data_datlas['COLONIA'].str.contains(r'\\N', na=False)]
data_datlas = data_datlas[~data_datlas['COLOR'].str.contains(r'\\N', na=False)]

Para poder realizar una mejor limpieza se requiere más tiempo y manejar una refencia en cuanto a las categorias que se buscan tene, por ejemplo, la lista de nombres de colonias por cada ciudad de Nuevo León.

In [26]:
data_datlas['CAUSA SINIESTRO'].value_counts()

CAUSA SINIESTRO
COLISION Y/O VUELCO           90445
FENOMENOS DE LA NATURALEZA      418
ATROPELLO                       345
INCENDIO RAYO O EXPLOSION        78
DAÑOS POR LA CARGA               49
TRANSPORTACION                   31
HUELGAS Y ALBOROTOS              26
Name: count, dtype: int64

In [27]:
data_datlas['TIPO VEHICULO'].value_counts()

TIPO VEHICULO
Auto             69216
Camión           19697
Camión Ligero     2365
Motocicleta        114
Name: count, dtype: int64

In [28]:
data_datlas['MODELO_VEHICULO'].value_counts()

MODELO_VEHICULO
2012    7124
2011    7095
2015    5877
2013    4846
2007    4759
        ... 
1976       1
1979       1
1952       1
1955       1
1987       1
Name: count, Length: 101, dtype: int64

In [29]:
data_datlas['NIVEL DAÑO VEHICULO'].value_counts()

NIVEL DAÑO VEHICULO
Bajo        50046
Sin daño    39754
Medio        1321
Alto          271
Name: count, dtype: int64

In [30]:
data_datlas['PUNTO DE IMPACTO'].value_counts()

PUNTO DE IMPACTO
Frontal                  22808
Trasero                  22792
Cofre                    13322
Costado der delantero     7497
Costado izq delantero     6195
Costado der trasero       5166
Costado izq trasero       4906
Costado der central       3717
Costado izq central       3389
Parte baja                 884
Toldo                      656
Tapa cajuela                60
Name: count, dtype: int64

In [31]:
print(data_datlas.columns)

Index(['LAT', 'LONG', 'CODIGO POSTAL', 'CALLE', 'COLONIA', 'CAUSA SINIESTRO',
       'TIPO VEHICULO', 'COLOR', 'MODELO_VEHICULO', 'NIVEL DAÑO VEHICULO',
       'PUNTO DE IMPACTO', 'AÑO', 'MES', 'DÍA NUMERO', 'DIA', 'HORA', 'ESTADO',
       'CIUDAD_APROXIMADA', 'DATE'],
      dtype='object')


In [32]:
print(data_datlas['AÑO'].value_counts().sort_index())
print("-----------------------------------")

AÑO
2016    41349
2017    34385
2018    15658
Name: count, dtype: int64
-----------------------------------


In [33]:
print(data_datlas['MES'].value_counts().sort_index())
print("-----------------------------------")

MES
1     9187
2     8955
3     8935
4     8664
5     9390
6     8903
7     5915
8     6218
9     6291
10    6331
11    6207
12    6396
Name: count, dtype: int64
-----------------------------------


In [34]:
print(data_datlas['DÍA NUMERO'].value_counts().sort_index())
print("-----------------------------------")

DÍA NUMERO
1     2848
2     3082
3     2963
4     3259
5     2913
6     2942
7     2963
8     3050
9     3055
10    2967
11    2926
12    3016
13    2958
14    3066
15    3265
16    3183
17    2900
18    3053
19    2938
20    3029
21    2933
22    3060
23    3121
24    2869
25    2784
26    2928
27    3182
28    3027
29    2868
30    2612
31    1632
Name: count, dtype: int64
-----------------------------------


In [35]:
print(data_datlas['DIA'].value_counts())
print("-----------------------------------")

DIA
VIERNES      15260
MARTES       14870
MIERCOLES    14579
JUEVES       14260
LUNES        13794
SABADO       12185
DOMINGO       6444
Name: count, dtype: int64
-----------------------------------


In [36]:
print(data_datlas['HORA'].value_counts().sort_index())
print("-----------------------------------")

HORA
0      809
1      569
2      449
3      363
4      309
5      354
6     1050
7     3820
8     5866
9     5457
10    5214
11    5441
12    5989
13    6942
14    6974
15    6395
16    6227
17    6298
18    6820
19    5868
20    4153
21    2837
22    1907
23    1281
Name: count, dtype: int64
-----------------------------------


In [37]:
print(data_datlas['ESTADO'].value_counts())
print("-----------------------------------")

ESTADO
NUEVO LEÓN    91392
Name: count, dtype: int64
-----------------------------------


In [38]:
## Vamos a eliminar el estado porque solo es 1 y no aporta mayor informacion
data_datlas = data_datlas.drop(['ESTADO'], axis=1)

In [39]:
print(data_datlas['CIUDAD_APROXIMADA'].value_counts().sort_index().iloc[:50])
print("Total de Ciudades:", len(data_datlas['CIUDAD_APROXIMADA'].unique()))
print("-----------------------------------")

CIUDAD_APROXIMADA
ABASOLO                      21
AGUALEGUAS                    1
ALLENDE                     929
ANAHUAC                      69
APODACA                    7446
AQUILES SERDÁN                2
ARAMBERRI                    10
ARBOLEDAS DEL VALLE           1
BUSTAMANTE                   13
CADEREYTA JIMENEZ          1615
CARMEN                       98
CENTRO DE LINARES             1
CERRALVO                     13
CHINA                        60
CIENEGA DE FLORES           353
COLONIA MIGUEL HIDALGO        2
DEL PASEO                     3
DEL VIDRIO                    1
DOCTOR ARROYO                52
DOCTOR GONZALEZ               6
EL CARMEN                     4
GALEANA                     169
GARCIA                     1445
GARZA GARCIA               7670
GENERAL BRAVO                67
GENERAL ESCOBEDO           4945
GENERAL TERAN                29
GENERAL ZARAGOZA              2
GENERAL ZUAZUA              248
GUADALUPE                 10555
HACIENDA LOS NOGALES  

In [40]:
# Vamos a reemplazar los valores repetidos de cada Ciudad
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('CARMEN', 'EL CARMEN')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('GARZA GARCIA', 'SAN PEDRO GARZA GARCIA')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('MIRADOR RESIDENCIAL', 'MONTERREY')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('HACIENDA LOS NOGALES', 'BUSTAMANTE')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('COLONIA MIGUEL HIDALGO', 'GUADALUPE')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('ARBOLEDAS DEL VALLE', 'LINARES')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('CENTRO DE LINARES', 'LINARES')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('CENTRO DE LINARES', 'LI')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('SAN RAFAEL', 'GALEANA')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('SAN ROBERTO', 'GUADALUPE')
data_datlas['CIUDAD_APROXIMADA'] = data_datlas['CIUDAD_APROXIMADA'].replace('RAMONES', 'LOS RAMONES')

# Eliminamos los valores que no son ciudades
data_datlas = data_datlas[data_datlas['CIUDAD_APROXIMADA'] != 'DEL VIDRIO']
data_datlas = data_datlas[data_datlas['CIUDAD_APROXIMADA'] != 'DEL PASEO']
data_datlas = data_datlas[data_datlas['CIUDAD_APROXIMADA'] != 'AQUILES SERDÁN']
data_datlas = data_datlas[data_datlas['CIUDAD_APROXIMADA'] != 'VALLE ORIENTE']
data_datlas = data_datlas[~data_datlas['CIUDAD_APROXIMADA'].str.contains(r'\\N', na=False)]

In [41]:
print(data_datlas['DATE'].value_counts().sort_index())

DATE
2016-01-01     50
2016-01-02     54
2016-01-03     60
2016-01-04     94
2016-01-05     94
             ... 
2018-06-26     80
2018-06-27     94
2018-06-28     98
2018-06-29    104
2018-06-30     83
Name: count, Length: 910, dtype: int64


In [44]:
data_datlas.reset_index(drop=True, inplace=True)

In [45]:
## De aquí podemos partir y usar está base de datos para continuar con otros análisis
data_datlas.head()

,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,CIUDAD_APROXIMADA,DATE
0,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018,1,25,JUEVES,10,JUAREZ,2018-01-25
1,25.718027,-100.249990,66480,AV. CONDUCTORES,CONDUCTORES INFONAVIT,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018,1,19,VIERNES,9,SAN NICOLAS DE LOS GARZA,2018-01-19
2,25.723590,-100.198780,67117,BOULEVARD ACAPULCO,REAL DE SAN MIGUEL,COLISION Y/O VUELCO,Auto,BLANCO,2000,Sin daño,Frontal,2018,1,29,LUNES,17,GUADALUPE,2018-01-29
3,25.708889,-100.246876,66480,PUERTO MARQUEZ 805,NUEVO MUNDO,COLISION Y/O VUELCO,Auto,AMARILLO,2011,Bajo,Frontal,2018,2,26,LUNES,2,SAN NICOLAS DE LOS GARZA,2018-02-26
4,25.684047,-100.213976,67160,AV. PLUTARCO ELIAS CALLES,LA JOYA INFONAVIT 2DO. SECTOR,COLISION Y/O VUELCO,Auto,AZUL,2011,Sin daño,Trasero,2018,1,1,LUNES,0,GUADALUPE,2018-01-01


In [46]:
data_datlas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91147 entries, 0 to 91146
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   LAT                  91147 non-null  float64       
 1   LONG                 91147 non-null  float64       
 2   CODIGO POSTAL        91147 non-null  object        
 3   CALLE                91147 non-null  object        
 4   COLONIA              91147 non-null  object        
 5   CAUSA SINIESTRO      91147 non-null  object        
 6   TIPO VEHICULO        91147 non-null  object        
 7   COLOR                91147 non-null  object        
 8   MODELO_VEHICULO      91147 non-null  object        
 9   NIVEL DAÑO VEHICULO  91147 non-null  object        
 10  PUNTO DE IMPACTO     91147 non-null  object        
 11  AÑO                  91147 non-null  int64         
 12  MES                  91147 non-null  int64         
 13  DÍA NUMERO           91147 non-

## 2.3 EDA (Exploratory Data Analysis)